In [1]:
#uncooment this line if gym is not installed
#!pip install gym

In [134]:
#uncooment this line if spacy is not installed
#!pip install -U spacy

In [2]:
import gym
# Importing libraries
import numpy as np
import random
import math
from collections import deque
import collections
import pickle

#for text processing
import spacy
import re
import pandas as pd
env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



#### There are 4 locations (labeled by different letters), and our job is to pick up the passenger at one location and drop him off at another. We receive +20 points for a successful drop-off and lose 1 point for every time-step it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions."

### Fetching Origing, Destination, and Time of Pickup from the sms data 

In [3]:
city_data = pd.read_csv("city.csv")
city_data

,location,mapping
0,dwarka sector 23,0
1,dwarka sector 21,1
2,hauz khaas,2
3,airport,3


In [4]:
city_locations = city_data['location'].values

In [5]:
city_locations = list(city_locations)
city_locations

['dwarka sector 23', 'dwarka sector 21', 'hauz khaas', 'airport']

In [6]:

def fetch_pickup_drop(text):
    
    city_locations = city_data['location'].values
    current_mapped_locations = city_locations
    
    
    
    index_of_to = text.rfind(' to ')
    index_of_for = text.rfind(' for ')
    index_of_from = text.rfind(' from ')
    
    if index_of_to > -1 and index_of_to > index_of_from:
        # example 1: 'Please book a cab from airport to hauz khaas at 3 PM'
        #example 2: #text = 'airport to hauz khaas at 6 PM'
        
        
        
        split1 = text[:index_of_to]
    
        for location in current_mapped_locations:
            if location in split1:
                origin = location
                break

        for location in current_mapped_locations:
            if location in text[index_of_to:]:
                destination = location
                break
    elif index_of_to > -1 and index_of_to < index_of_from:
        # example 1: 'Please book a cab from airport to hauz khaas at 3 PM'
        #example 2: #text = 'airport to hauz khaas at 6 PM'
        
        split1 = text.split(' from ')
    
        for location in current_mapped_locations:
            if location in split1[1]:
                origin = location
                break

        for location in current_mapped_locations:
            if location in split1[0]:
                destination = location
                break
    elif index_of_to == -1 and index_of_for < index_of_from:
        #example 1: #text = 'Please book a cab for hauz khaas from dwarka sector 21 at 1 PM'
        split1 = text.split(' from ')
    
        for location in current_mapped_locations:
            if location in split1[1]:
                origin = location
                break

        for location in current_mapped_locations:
            if location in split1[0]:
                destination = location
                break

    match = re.search("((([1-9])|(1[0-2]))\s(A|P|a|p)(M|m))", text)
    time_of_pickup = match.group()
    
    return [origin, destination, time_of_pickup]




In [7]:
org_data = pd.read_csv("org_df.csv")
org_data

,origin,dest,time
0,airport,hauz khaas,3 PM
1,airport,hauz khaas,6 PM
2,hauz khaas,dwarka sector 23,1 PM
3,airport,hauz khaas,1 AM
4,airport,dwarka sector 21,10 PM
...,...,...,...
995,airport,dwarka sector 23,2 AM
996,dwarka sector 21,dwarka sector 23,2 PM
997,hauz khaas,dwarka sector 21,5 AM
998,airport,dwarka sector 23,6 PM


In [8]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


## Summing up the Q-Learning Process
Breaking it down into steps, we get

Initialize the Q-table by all zeros.

Start exploring actions: 

For each state, select any one among all possible actions for the current state (S).

Travel to the next state (S') as a result of that action (a).

For all possible actions from the state (S') select the one with the highest Q-value.

Update Q-table values using the equation.

Set the next state as the current state.

If goal state is reached, then end and repeat the process.


## Exploiting learned values
After enough random exploration of actions, the Q-values tend to converge serving our agent as an action-value function which it can exploit to pick the most optimal action from a given state.

There's a tradeoff between exploration (choosing a random action) and exploitation (choosing actions based on already learned Q-values). We want to prevent the action from always taking the same route, and possibly overfitting, so we'll be introducing another parameter called ϵ "epsilon" to cater to this during training.

Instead of just selecting the best learned Q-value action, we'll sometimes favor exploring the action space further. Lower epsilon value results in episodes with more penalties (on average) which is obvious because we are exploring and making random decisions.

In [9]:
#Initialize Q_table
import numpy as np
#write your code here

q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [10]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

##Write your code here
# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")


np.save("./q_table.npy", q_table)

Episode: 100000
Training finished.

CPU times: user 1min 12s, sys: 15.9 s, total: 1min 28s
Wall time: 1min 10s


In [11]:
#Load trained q_table for evaluation

q_table = np.load("./q_table.npy")

In [12]:
def create_loc_dict(city_df):
    loc_dict = {}
    ## Create dictionary example, loc_dict['dwarka sector 23] = 0
    df_size = city_df.shape[0]
    for i in range(df_size):
        city_name = city_data.iloc[[i]]['location'].values[0]
        map_value = city_data.iloc[[i]]['mapping'].values[0]
        loc_dict[city_name] = map_value
    
    return loc_dict

In [13]:
def check_pick_up_drop_correction(pick_up, drop, line_num):
    
    fp = open("./sms.txt", "r")
    line = f.readline()
    for i, line in enumerate(fp):
        
        if i == line_num - 1:
            fetch = fetch_pickup_drop(line)
        elif i >= line_num:
            break
    
    fp.close()
    original_origin = fetch[0]
    original_destination = fetch[1]
    
    if original_origin == pick_up and original_destination == drop:
        return True
    else:
        return False
    

In [14]:
"""Evaluate agent's performance after Q-learning"""

# 1) We need to take text drom "sms.txt" and fetch pickup and drop from it.
# 2) Generate the random state from an enviroment and change the pick up and drop as the fetched one
# 3) Evaluate you q_table performance on all the texts given in sms.txt.
# 4) Have a check if the fetched pickup, drop is not matching with original pickup, drop using orig.csv
# 5) If fetched pickup or/and drop does not match with the original, add penality and reward -10
# 6) Calculate the Total reward, penalities, Wrong pickup/drop predicted and Average time steps per episode.

total_epochs, total_penalties, total_reward, wrong_predictions = 0, 0, 0, 0


count = 0
time_list = []

f = open("./sms.txt", "r")

num_of_lines = 1000
city = pd.read_csv("./city.csv")


loc_dict = create_loc_dict(city)

line_num = 0

# read all lines
all_lines = f.readlines()

for _ in range(num_of_lines):
    
    text = all_lines[line_num]
    orig,dest,pick_up_time = fetch_pickup_drop(text)
    passenger_index = loc_dict[orig]
    destination_index = loc_dict[dest]
    
    state = env.reset()
    state = env.encode(0, 0, passenger_index, destination_index) # (taxi row, taxi column, passenger index, destination index)
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1
            count += 1

        epochs += 1
        
    line_num += 1
    
    

    total_penalties += penalties
    total_epochs += epochs
    wrong_predictions = count
    total_reward = reward


print(f"Results after {num_of_lines} episodes:")
print(f"Average timesteps per episode: {total_epochs / num_of_lines}")
print(f"Average penalties per episode: {total_penalties / num_of_lines}")
print(f"Total number of wrong predictions", wrong_predictions)
print("Total Reward is", total_reward)

Results after 1000 episodes:
Average timesteps per episode: 14.179
Average penalties per episode: 0.244
Total number of wrong predictions 244
Total Reward is 20
